In [1]:
from TwitterSearch import *
import string
from string import digits
import Stemmer
import re
import pandas as pd
import numpy as np

In [5]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM


Using TensorFlow backend.


In [94]:
tweets=[]
try:
    tso = TwitterSearchOrder() 
    tso.set_keywords(['времяпервых','ВремяПервых']) 
    tso.set_language('ru') # russian tweets only
    tso.set_include_entities(False) # and don't give us all those entity information

    ts = TwitterSearch(
        consumer_key = 'XfyrnpYx2jgkvqVMO28Dl4vro',
        consumer_secret = 'DNU3vVeboWDmRIC7PBi08j2hpK05mhiw5cYHRIJZqOUIKQ0hob',
        access_token = '410867738-Yeyao2EBTjBVxruKM9Zw7MrvKmmkRxmbeHm1GGlx',
        access_token_secret = 'sxxY0nVKVt9o0JbZGojJAEG6yGHHIKykOaDImOtGPAWPq'
     )

     
    for tweet in ts.search_tweets_iterable(tso):
        tweets.append('%s'%(tweet['text']))

except TwitterSearchException as e: # take care of all those ugly errors if there are some
    print(e)

In [95]:
tweets=list(set(tweets))

In [96]:
def clear_tweets(tweets):
    stemmer = Stemmer.Stemmer('russian')
    tags=['времяпервых','_','__']
    remove_digits = str.maketrans('', '', digits)
    remove_latins=str.maketrans('','','abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ')
    
    #clear tweets from digits, punctuation, latin words. Just splitted words
    clr_tweets=[]
    for s in tweets:
        s=s.translate(remove_digits)
        s=s.translate(remove_latins)
        s=s[:s.find('https://')]
        a=re.sub('\W',' ', s).split()
        for i in a:
            if i.lower() in tags:
                a.remove(i)
        #a=stemmer.stemWords(a) 
        clr_tweets.append(a)
        
    return (clr_tweets)


In [97]:
#make a list from list of lists
def listmerge(lstlst):
    all=[]
    for lst in lstlst:
        all.extend(lst)
    return all


In [34]:
#make a new list where tweets now are sequences of numbers 
def tweets_to_numb(clr_tweets, dic):
    tweets_to_numb=[]
    for i in range (len(clr_tweets)):
        a=[dic[s] for s in clr_tweets[i]]
        tweets_to_numb.append(a)
    
    return (tweets_to_numb)

In [98]:
clr_tweets=clear_tweets(tweets)

In [11]:
#load train sets of positive and negative tweets
positive=pd.read_csv('positive.csv', encoding='utf-8',sep=';', header=None,usecols=[3,4])
negative=pd.read_csv('negative.csv', encoding='utf-8',sep=';', header=None,usecols=[3,4])



In [100]:
pos_tweets=clear_tweets(list(positive.iloc[:,0]))
neg_tweets=clear_tweets(list(negative.iloc[:,0]))

In [80]:
#Make a dict. Every word gets its number
all_words=listmerge(clr_tweets+pos_tweets+neg_tweets)
dic=dict(zip(all_words,range(len(all_words))))

KeyboardInterrupt: 

In [101]:
all_words=listmerge(clr_tweets+pos_tweets+neg_tweets)

In [111]:
len(all_words)

2498110

In [23]:
#Now we have sequences of numbers as tweets
posit=tweets_to_numb(pos_tweets,dic)
negat=tweets_to_numb(neg_tweets,dic)
X_test=tweets_to_numb(clr_tweets,dic)

In [24]:
X_train=posit+negat
y_train=[1]*len(posit)+[0]*len(negat)

In [25]:
max_features = len(all_words)
maxlen = 32
batch_size = 32
model = Sequential()
model.add(Embedding(max_features,128))
model.add(LSTM(64, return_sequences=True))
model.add(LSTM(64))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              class_mode="binary")


In [26]:
for i in X_train:
    i = np.array(i) 
for i in X_test:
    i=np.array(i)
    
y_train=np.array(y_train)

In [27]:
x_train=sequence.pad_sequences(X_train, maxlen=maxlen)
x_test=sequence.pad_sequences(X_test, maxlen=maxlen)

In [28]:
model.fit(
    x_train, y_train,
    batch_size=batch_size, 
    epochs=1
    #show_accuracy=True
)

result = model.predict_proba(x_test)

/Applications/anaconda/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:2124: UserWarning: Expected no kwargs, you passed 1
kwargs passed to function are ignored with Tensorflow backend
  warnings.warn('\n'.join(msg))


Epoch 1/1
 96/120 [=======================>......] - ETA: 0s

In [29]:
dict(zip(tweets,result))

{'"Время первых" - добротно. Очень добротно #времяпервых': array([ 0.78492516], dtype=float32),
 '"Время первых"-потрясающий фильм! Держит в напряжении до конца!Его стоит посмотреть!!! Не пожалеете! #космос #фильм #ВремяПервых #отдых': array([ 0.93038428], dtype=float32),
 '#ВремяПервых\nПросто офигенный фильм, ни капли не жалею, что сходила': array([ 0.13313496], dtype=float32),
 '#ВремяПервых - 10 литров слёз из 10': array([ 0.30430153], dtype=float32),
 '#ВремяПервых - лучший российский фильм, который я когда-либо смотрела': array([ 0.79287905], dtype=float32),
 '#ВремяПервых - очень хорошо': array([ 0.70111781], dtype=float32),
 '#ВремяПервых : очень понравился, волнительно, интересно, патриотично, правдиво. Советую идти!': array([ 0.7618829], dtype=float32),
 '#ВремяПервых действительно сильно. понравилось 👐🏻': array([ 0.72896761], dtype=float32),
 '#ВремяПервых для нас ПРОШЛО?.. Новое видео, сегодня есть неплохой повод подумать:\n\nhttps://t.co/jtH547gLer https://t.co/FhC4GkfaF7'

In [126]:
dictionary={'"Время первых" - добротно. Очень добротно  ':  0.78492516 ,
 '"Время первых"-потрясающий фильм! Держит в напряжении до конца!Его стоит посмотреть!!! Не пожалеете! #космос #фильм   #отдых':  0.93038428 ,
 '  Просто офигенный фильм, ни капли не жалею, что сходила':  0.13313496 ,
 '  - 10 литров слёз из 10':  0.30430153 ,
 '  - лучший российский фильм, который я когда-либо смотрела':  0.79287905 ,
 '  - очень хорошо':  0.70111781 ,
 '  : очень понравился, волнительно, интересно, патриотично, правдиво. Советую идти!':  0.7618829 ,
 '  действительно сильно. понравилось 👐🏻':  0.72896761 ,
 '  для нас ПРОШЛО?.. Новое видео, сегодня есть неплохой повод подумать:   jtH547gLer  FhC4GkfaF7':  0.91335166 ,
 '  хорошее кино, но голливудская пафосная музыка выбивала из атмосферы.':  0.73788005 ,
 '  хороший фильм с напряжённой атмосферой. Ещё один русский годный фильм! Даю оценку 9/10. #КраткОбзор  O0h5QMvpzu':  0.8463921 ,
 '  это что-то ❤':  0.45554388 ,
 '  посмотрели. В кой то веки не зря потратила деньги на русский фильм.':  0.61710507 ,
 '#КраткОбзор   Миронов реабилитировался в моих глазах после утомленных солнцем 2. 8 Хабенских из 10  GDGQ8fxpla':  0.54611593 ,
 '   Фильм очень хороший!!!  Cq3E3iGouF':  0.82250398 ,
 '  - хороший сюжет для русского фильма, к созданию подошли ответственно.Только с хронометражом затянули.8/10 #краткобзор':  0.8419624 ,
 '  Ах, чудо, что за фильм! Побольше бы таких!':  0.67627704 ,
 '  Время поднять свою голову вверх в небо .':  0.72916532 ,
 '  нереальный фильм потеряла дар речи  в голове много мыслей о жизни появилось всем советую 🌍':  0.85676706 ,
 '  прекрасный фильм. Рекомендую для просмотра всем возрастам #кинотеатр  YbCBWbG6Uj':  0.91638285 ,
 '  умилил пафосными монологами и неестественными каламбурными фразами, из которых состояло 90% диалогов. Да это же прям как я пишу':  0.75269395 ,
 '  фильм хочетсч сохранить цитатами и кадрами, обязательно буду пересматривать, а пока...  19Slw57m7y':  0.902022 ,
 '(Я кажется знаю к чему относилась фраза "ну все, мандец космосу"    oBTGYQbBcT':  0.74122226 ,
 '60-е. Разгар холодной войны.  Две супердержавы, СССР и США бьются за первенство в космической гонке.   Пока СССР вп…  HhsVXdDrIn':  0.67935765 ,
 '@RybkinaTatiana @GlavaBogorod @bogadmzsocial @luneva_na Коллективные посещения школьниками кинопоказа  …  LdV6IFKkza':  0.47122812 ,
 '@medinskiy_vr пардон, но те фильмы, которые вы тут показали - убогая поделка и дно. Не могли что-то получше прикреп…  9SVi9s4rgf':  0.69738877 ,
 '@syunusow Да Защитники собрали немногим меньше  , о чем еще говорить? :/':  0.70767367 ,
 '@turist_07 ПосмотрюА я ходила на  🚀Под впечатлением🛰🌌🌏,в отличие от твоего чертановского-#Притяжение👽…  hGpepyv8VW':  0.68242818 ,
 '@uNkinder_ У   сборы в России 6 млн. $, а у #форсаж8, который иза первого вышел на неделю позже уже 14 м…  yYhmcVofjS':  0.48280087 ,
 '@yandex 🤑Мне заплатят?😬    hKoGb3nvC8':  0.43376634 ,
 'Maxim Novikov 〽: Сначала #никак , потом оч оч понравился #хорошийфильм   #фильм #рекомендую……  lVztgDdsA1':  0.62507993 ,
 ' 4ex_one: Повзрослел - это когда идешь на #Форсаж, но покупаешь билеты на   и остаешься доволен своим выбором. #КраткОбзор':  0.74972898 ,
 ' Calat_spb: Российское кино все лучше и лучше! "Время первых" - тому пример. #киновечер  ':  0.80849403 ,
 ' DLytkinmsk: Миронов,Хабенский и Ильин держали внимание с первой секунды и до самого выхода из зала.Очень рекомендую, обязательно посмот…':  0.71354443 ,
 ' EKarlova: ВРЕМЯПЕРВЫХ-эти 2немолодых,усталых парня,случайно попавших в космонавты,чуть не ставших жертвой тупого режима не позовут МОЛО…':  0.39729148 ,
 ' L_One_Mars: Премьера видео! L\'ONE - "Время Первых"  h0Sm68oPsg   ':  0.79594702 ,
 ' MC_Azimut: @RybkinaTatiana @GlavaBogorod @bogadmzsocial @luneva_na Коллективные посещения школьниками кинопоказа   продолжае…':  0.72580421 ,
 ' MC_Azimut: @RybkinaTatiana @GlavaBogorod @luneva_na @bogadmzsocial Коллективное посещение показа   школьниками города #Богор…':  0.59992832 ,
 ' ResidentRostov: Пошли в кино    (@ Золотой Вавилон in Ростов-на-Дону, Ростовская обл.  8KQSDZiR8v  …':  0.64740694 ,
 ' black_star_ru: ПРЕМЬЕРА КЛИПА @L_One_Mars -    Смотри прямо сейчас:  Z8RULP6ZOe':  0.78468102 ,
 ' cuprum_1:   #Россия #фильм #  Действительно ли в фильм с привкусом Левиафанщины?':  0.70356065 ,
 ' ormvdrf: Удостоверение пришлось сдать   2poVuxwfyJ #мвд #ормвд #ЩИТиМЕЧ #Россия #Пермь #Космонавты #Космос #СССР #ВремяПерв…':  0.70731896 ,
 ' taki_yagadka: Шыдэвр¡  ':  0.52351439 ,
 'Ближайшие премьеры АРТа!   #форсаж8 #кухня #урфинджюс #сфера #моядевушкамонстр…  SgoSfo299a':  0.65349436 ,
 'В последнее время российский кинематограф радует меня все больше.    офигенный фильм👍🏻':  0.88997602 ,
 'В современном мире мы не часто встречаем героев! Алексей Леонов яркий пример героизма российского народа. Смотрите #фильм    !':  0.94997793 ,
 'ВРЕМЯПЕРВЫХ-эти 2немолодых,усталых парня,случайно попавших в космонавты,чуть не ставших жертвой тупого режима не по…  m2bFLBLivr':  0.28027523 ,
 'Весь зал аплодировал.  ':  0.45866367 ,
 'Впервые жалею о том, что среди населения уже сложилось стереотипно-пренебрежительное отношение к отечественному кин…  VTHSapHUvs':  0.25255838 ,
 'Время первых не прошло   Фильм смотрится на одном дыхании. Картина рассказывает о первом выходе в открытый космос.…  BdaqYdYTVZ':  0.91576928 ,
 "Время первых, время смелых... #L'One   #минск #выходные @ Мир Фитнеса  cZik1EWpN9":  0.90851957 ,
 'Время поднять свою голову вверх-вверх, в небо  ':  0.71057624 ,
 'Вроде как этот фильм все рекомендуют, и мы решили сходить посмотреть! #ночь    #кино…  6zleelrQEl':  0.91500533 ,
 'Духоподъемное кино про советскую космонавтику. К просмотру рекомендую.  …  7xVlzbPzco':  0.7514416 ,
 'Есть особые люди они не боятся мечтать.  ':  0.50203693 ,
 'Женя Миронов, я тебя люблю  ':  0.65410364 ,
 'За 2 нед. до старта взрывается корабль. Времени на выявление причин нет. И пусть риски огромны, мы не можем уступит…  ORm1UpBhzE':  0.46896234 ,
 'Знаете, почему   такой прекрасный фильм? Потому что по-настоящему наш, не натянутый на чужой стандарт. Можем, когда хотим и верим':  0.7486921 ,
 'Каждое новое утро, не хочется быть одному тут. Тут без шуток никак, каждый день по новым странным маршрутам.  ':  0.39361361 ,
 'Как-то так #ЧтоДелатьКонсалт   #Консультант #Консалт #работа #праздник #победа…  iU78VoJy4Q':  0.63086438 ,
 'Качественный, эмоциональный и прекрасный фильм. Мужество, сила и патриотизм в чистом виде    #Миронов…  UmUjM2MseT':  0.75153285 ,
 'Космическая #драма "Время первых": не надо стесняться гордиться за #СССР #кино #фильм   #космос #Леонов  d998CcnoCR':  0.91445893 ,
 'Круто видеть @L_One_Mars везде!🚀 Есть особые люди, они не бояться мечтать! #20мая #берисвое    QawUoYBBI9':  0.59489244 ,
 'Кто с нами с Санькой сегодня в 19:40 на   в #люксормитино ? 😀🎥🎬📽🔭🚀🌌🌍  2Om3ki4r5s':  0.64255047 ,
 'Кто уже посмотреть "Время первых"? Ждём ваши отзывы о фильме!  jfPlLfPigQ    RuTACVYqSp':  0.92565739 ,
 'Маму сводила на  . Фильм сильный, понравился очень и временами просто до слез.':  0.58760178 ,
 'Миронов,Хабенский и Ильин держали внимание с первой секунды и до самого выхода из зала.Очень рекомендую, обязательн…  FqEDfynVX7':  0.71451223 ,
 'Мне думается, кто-то где-то наверняка уже строчит фикло.    #соринотсори  mnbcz3G8wC':  0.69339705 ,
 'Можно было бы пошутить, что даже в космос мы летали "через жопу", но слишком грустно все это. Стрёмно. А кино хорошее.   ':  0.36603972 ,
 'Моя оценка фильму   производства России в 2017 году 9 из 10 .Жаль Борьба с фильмом #Форсаж обернулась тем же местом к':  0.4239701 ,
 'Мы ходили на "время первых"    Про космонавтов Захватывающе':  0.7619971 ,
 'НАША ИСТОРИЯ    /   / 50 лет назад в СССР простились с летчиком-космонавтом Владимиром Комаровым. &lt;br..  HGn0TGsoH6':  0.5870682 ,
 'Наконец-то годнота!  ':  0.83785367 ,
 'Никогда еще, смотря фильм столько не рыдал  ':  0.25430432 ,
 'Ну наконец-то вышел достойный российский фильм 😱 Отличное кино без лишнего пафоса, которое вызывает гордость Спасибо за него  ':  0.88880271 ,
 'Ну что. И вправду хороший фильм.    0KlqQxTjXB':  0.76389742 ,
 'Обязательно сходите на этот фильм в кинотеатр, не пожалеете!    F9DFOAsIOh':  0.47011626 ,
 'Опытный военный летчик Павел Беляев и его напарник Алексей Леонов, – два человека, готовые шагнуть в неизвестность.…  yJAO15JIMW':  0.74448866 ,
 'Отличное интервью в апрельском номере Esquire    #поднимиголову #евгениймиронов…  GYUk2LjfW7':  0.68937337 ,
 'Очень понравилось   ну могут же снимать, если захотят':  0.78145635 ,
 'Первый раз в жизни и , наверное,последний мы были на фильме,после которого зал хлопал! Вы понимаете ??ХЛОПАЛ!!  ':  0.78581059 ,
 'По педагогике задали сделать проект до 31го мая. Выбрал тему - возвращение астрономии в российские школы.  ':  0.57082331 ,
 'Повзрослел - это когда идешь на #Форсаж, но покупаешь билеты на   и остаешься доволен своим выбором. #КраткОбзор':  0.74972898 ,
 'Попалось на глаза. Купаться захотелось.   #тольятти #волга #капылово  gLQ01D2lI6':  0.44285017 ,
 'Посмотрела фильм   который основан на реальных событиях, исторический и очень атмосферный':  0.81028843 ,
 'Посмотрела фильм   который основан на реальных событиях, исторический и очень атмосферный.':  0.81028843 ,
 'Потрясающий фильм, с любовью к небу, мужеству и упорству.  ':  0.81589919 ,
 'Пошли в кино    (@ Золотой Вавилон in Ростов-на-Дону, Ростовская обл.  8KQSDZiR8v  UZ3ILivkAr':  0.64740694 ,
 'Прилетело неожиданно, но приятно! Спасибо @l_one_mars   #pillseller @ Changsha, China  B1F7SPD077':  0.93824577 ,
 'РАСПИСАНИЕ КИНОТЕАТРА📺📹 "НОВАЯ ЗАРЯ" С 20.04.17г. ПО 26.04.17г. #форсаж8 #ихдорогирасходятся  …  4Z7EZuJVSb':  0.77695304 ,
 'РАСПИСАНИЕ КИНОТЕАТРА📺📹 "НОВАЯ ЗАРЯ" С 20.04.17г. ПО 26.04.17г. #форсаж8 #ихдорогирасходятся  …  j1pfSLYC5C':  0.77695304 ,
 'Решили с tosi17101986 сходить в кино в кинотеатр #киномир в трц #эдем на   @…  K6GClGkO9V':  0.61624527 ,
 'Российский кинематограф сделал действительно шикарное кино!  ':  0.87939531 ,
 'Российское кино все лучше и лучше! "Время первых" - тому пример. #киновечер  ':  0.80849403 ,
 'С любимым Дедом сходили в кино на Время Первых в 3D! Отличный фильм! #DeeOnline    #КосмосЗаНас  V5qSCN0hqH':  0.89976412 ,
 'Сегодня наконец-то посмотрели  . Не впечатлил так, как ожидали, но такие фильмы однозначно стоит смотреть! И снимать 1️⃣🆙🚀🌎':  0.9064762 ,
 'Сегодня, я ходил вместе со своим классом,ну и с другими тоже классами на ,,Время первых".Весь зал аплодировал,и я тоже  ':  0.67862236 ,
 'Сколько эмоций всего за 2,5 часа🙏🏻👨🏻\u200d🚀♥️  ':  0.55866599 ,
 'Смотришь   и думаешь "как космонавты вообще помещались в столь маленьком пространстве в своих-то объёмных костюмах?"':  0.72118962 ,
 'Снимки Роскосмоса могут стать идеальной обложкой для какого-нибудь модного альбома  ':  0.70282352 ,
 'Спасибо, поревел 👍 Ну а так, этот фильм действительно годный   ':  0.82297283 ,
 'Сходил на    Первый нормальный отечественный #фильм Неудивительно, что весь зал апплодировал':  0.70807678 ,
 'Сходила сегодня в кино Смотрела    И скажу вам, что это прекрасный фильм!':  0.95375919 ,
 'Сходили на   Это. Было. Очень. Сильно':  0.79361212 ,
 'Съездила в Адлер, посмотрела   Затронул до глубины души, после окончания аплодировали все! Сильно, патриотично, проникновенно! ❤️':  0.70098495 ,
 'Теперь можно и в космос 😌🚀  ':  0.80217224 ,
 'То #чувство, #когда #борщ попал космонавту в #глаз. #космос #космонавт   #кино  KdUhgciuVp':  0.30130476 ,
 'Ты ищешь примеры людей, что падали, но поднимались,  Падали, да, но не сдались.   ':  0.32312593 ,
 'Удостоверение пришлось сдать   2poVuxwfyJ #мвд #ормвд #ЩИТиМЕЧ #Россия #Пермь #Космонавты #Космос #СССР…  pFaW6CTQkH':  0.70926154 ,
 'Фильм "ВРЕМЯ ПЕРВЫХ" просто бомба🍿💥😻  ':  0.86416191 ,
 'Ходила сегодня на   , какой же замечательный фильм Хоть я и не люблю советские фильмы, но этот посмотрела с удовольствием':  0.76498657 ,
 'Хоть и концовка известна,сидишь как на иголках, мы просто крупицы  ':  0.70955741 ,
 'Хотя и остались некоторые вопросы, ставлю 10/10  #суперрецензия    Были бы современные Российские фильмы все такими🙏':  0.77905172 ,
 'Хочется стереть себе память и посмотреть   ещё раз.❤️':  0.54982251 ,
 'Шыдэвр¡  ':  0.52351439 ,
 'Эмблему Первого канала на фоне Вселенной показывали чуть дольше, чем логотипы всех остальных спонсоров фильма  ':  0.86947674 ,
 'Я горд, что родился в #СССР, я горд, что я #Русский!!!    mig8fcz889  sUqlHTUHYy':  0.72501516 ,
 
 'честно,ожидал очередной пафосной бекмамбетовщины,но получился один из лучших росс.фильмов за последнее время   #рекомендую':  0.80745298 }


In [139]:
true_list=[1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1, 1,1,0,1,0,0,0,0,0,1,0,0,1,1,1,1,0,0,0,0,0,0,1,0,1,0,1,1,0,1,0,1,1,1,1,1,1,1,0,1,0,0,1,1,1,0,0,1,1,0,0,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,0,0,0,1,1,1,1,1,1,0,0,1,1,1,1, 1,0,0,0,0,1,1,1,1,1,1,0,1,1
]

In [151]:
accuracy=[]
recall=[]
precision=[]
for k in range (30, 80, 5):
    new_list=[]
    tp=0
    acc=0
    fn=0
    fp=0
    for item in list(dictionary.values()):
        if item>=k/100:
            new_list.append(1)
        else:
            new_list.append(0)
    for i in range (len(true_list)-1):
        if true_list[i]==new_list[i]:
            acc+=1
            if true_list[i]==1:
                tp+=1
        elif true_list[i]==1 and new_list[i]==0:
            fn+=1
        else:
            fp+=1
    accuracy.append(acc/len(true_list))
    recall.append(tp/(tp+fp))
    precision.append(tp/(tp+fn))
